In [1]:
import pandas as pd
import numpy as np
#from logs import get_logger
import src.article_relevance as ar

/Users/sedv8808/HT-Data/UWisc/article-relevance/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Notebook to Prepare 3 Training PQ Files

## Fixed Neotoma & Pollen files from CrossRef

In [2]:
neotoma_fixed = pd.read_csv('data/raw/neotoma_crossref_fixed.csv')
pollenDF = pd.read_csv('data/raw/pollen_doc_labels.csv')

Extracting DOIs and extracing data from CrossRef

In [3]:
neotomaCrossRef = ar.crossRefQuery(neotoma_fixed['doi'].unique().tolist())
pollenCrossRef =  ar.crossRefQuery(pollenDF['doi'].unique().tolist())

/Users/sedv8808/HT-Data/UWisc/article-relevance/src/article_relevance/crossRefQuery.py:25: Warning: DOI 10.1016/j.quascirev.2016.09.023. not found in CrossRef
  warnings.warn(warning_msg, category=Warning)
/Users/sedv8808/HT-Data/UWisc/article-relevance/src/article_relevance/crossRefQuery.py:25: Warning: DOI fiedel, s.j. 2018. the spore conundrum: does a dung fungus decline signal humans' arrival in the eastern united states?. quaternary international 466:247e255. [doi: doi.org/10.1016/j.quaint.2015.11.130] not found in CrossRef
  warnings.warn(warning_msg, category=Warning)
/Users/sedv8808/HT-Data/UWisc/article-relevance/src/article_relevance/crossRefQuery.py:25: Warning: DOI widga, c., s.n. lengyel, j. saunders, g. hodgins, j.d. walker, and a.d. wanamaker. 2017. late pleistocene proboscidean population dynamics in the north american midcontinent. boreas 46(4):772-782. [doi: 10.1111/bor.12235] not found in CrossRef
  warnings.warn(warning_msg, category=Warning)
/Users/sedv8808/HT-Data

Concatenate the two dataframes.

In [4]:
df = pd.concat([neotomaCrossRef, pollenCrossRef])
df = df.reset_index(drop=True)

In [5]:
df.head(2)

,DOI,title,subtitle,author,subject,abstract,container-title,language,published,publisher,URL,CrossRefQueryDate
0,10.1016/j.palaeo.2007.07.008,[Vegetation and fire history during the last 1...,[],"[{'given': 'Vera', 'family': 'Markgraf', 'sequ...","[Paleontology, Earth-Surface Processes, Ecolog...",,"[Palaeogeography, Palaeoclimatology, Palaeoeco...",en,"{'date-parts': [[2007, 10]]}",Elsevier BV,http://dx.doi.org/10.1016/j.palaeo.2007.07.008,2023-09-12 13:50:32.389184
1,10.1016/j.quascirev.2015.03.014,[The origin of grasslands in the temperate for...,[],[{'ORCID': 'http://orcid.org/0000-0001-9605-82...,"[Geology, Archeology, Archeology, Ecology, Evo...",,[Quaternary Science Reviews],en,"{'date-parts': [[2015, 5]]}",Elsevier BV,http://dx.doi.org/10.1016/j.quascirev.2015.03.014,2023-09-12 13:50:32.769019


In [6]:
#df.to_csv('data/raw/data_full.csv',  index=False)

In [ ]:
#df = pd.read_csv('data/raw/data_full.csv')
#df.head(2)

In [7]:
preprocessedDF = ar.dataPreprocessing(df)

2023-09-12 14:00:12,415 - dataPreprocessing.py:22 - dataPreprocessing - INFO - Prediction data preprocessing begin.
2023-09-12 14:00:12,453 - dataPreprocessing.py:70 - dataPreprocessing - INFO - Running article language imputation.
2023-09-12 14:00:12,463 - dataPreprocessing.py:86 - dataPreprocessing - INFO - 151 articles require language imputation
2023-09-12 14:00:12,464 - dataPreprocessing.py:88 - dataPreprocessing - INFO - 5 cannot be imputed due to too short text metadata(title, subtitle and abstract less than 5 character).
2023-09-12 14:00:13,845 - dataPreprocessing.py:96 - dataPreprocessing - INFO - Missing language imputation completed
2023-09-12 14:00:13,851 - dataPreprocessing.py:97 - dataPreprocessing - INFO - After imputation, there are 14 non-English articles in total excluded from the prediction pipeline.
dup cols
Index([], dtype='object')


In [8]:
preprocessedDF.head(2)

,CrossRefQueryDate,DOI,URL,abstract,author,container-title,language,published,publisher,subject,subtitle,title,titleSubtitleAbstract,validForPrediction
0,2023-09-12 13:50:32.389184,10.1016/j.palaeo.2007.07.008,http://dx.doi.org/10.1016/j.palaeo.2007.07.008,,"[{'given': 'Vera', 'family': 'Markgraf', 'sequ...","[Palaeogeography, Palaeoclimatology, Palaeoeco...",en,"{'date-parts': [[2007, 10]]}",Elsevier BV,"[Paleontology, Earth-Surface Processes, Ecolog...",,Vegetation and fire history during the last 18...,vegetation and fire history during the last 18...,1
1,2023-09-12 13:50:32.769019,10.1016/j.quascirev.2015.03.014,http://dx.doi.org/10.1016/j.quascirev.2015.03.014,,[{'ORCID': 'http://orcid.org/0000-0001-9605-82...,[Quaternary Science Reviews],en,"{'date-parts': [[2015, 5]]}",Elsevier BV,"[Geology, Archeology, Archeology, Ecology, Evo...",,The origin of grasslands in the temperate fore...,the origin of grasslands in the temperate fore...,1


Save this as 1 parquet file.

In [13]:
preprocessedDF.to_parquet('data/raw/processedDF.parquet', engine='fastparquet', compression='snappy')

In [14]:
preprocessedDF.columns[preprocessedDF.columns.duplicated()]


Index([], dtype='object')

## Annotation Data File

We need a 2nd file that contains the following info:
```python
['DOI', 'Annotation', 'Annotator', 'Annotation_Date']
```

In the `pollen.csv` file Dr. Goring has done already some manual annotation of whether a particular article would belong to Neotoma or not. All files in the `neotoma.csv` file belong to Neotoma. 

In a different setting, this annotations would be made using the **Data Review Tool** and stored to complement the parquet file, for now, I am going to do a JSON

In [ ]:
from datetime import datetime
annotation_cols = ['DOI', 'annotation', 'annotator', 'annotationDate']

In [ ]:
neotoma_fixed['DOI'] = neotoma_fixed['doi']
neotoma_fixed['annotation'] = 'Neotoma'
neotoma_fixed['annotator'] = 'Simon J. Goring'
neotoma_fixed['annotationDate'] = datetime.now()
neotoma_fixed['annotationDate'] = neotoma_fixed['annotationDate'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
neotomaAnnotation = neotoma_fixed[annotation_cols]

In [ ]:
pollenDF['DOI'] = pollenDF['doi']
pollenDF['annotation'] = pollenDF['Label']
pollenDF['annotator'] = pollenDF['Profile']
pollenDF['annotationDate'] = pollenDF['Timestamp']
#pollenDF['annotationDate'] = pollenDF['annotationDate'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
pollenAnnotation = pollenDF[annotation_cols]

In [ ]:
fullAnnotation = pd.concat([neotomaAnnotation, pollenAnnotation])
fullAnnotation = fullAnnotation.reset_index()

In [ ]:
fullAnnotation.to_parquet('data/parquet/neotomaAnnotation.parquet', index=False)

## Prediction Parquet

The Prediction PQ file will contain the following columns:
```python
['DOI', 'prediction', 'predict_proba', 'model_metadata', 'prediction_date']
```

Embed the data using the provided functions (this has been previously trained).

In [ ]:
embeddedData = ar.addEmbeddings(preprocessedDF, 'titleSubtitleAbstract')

Do the predictions using the AWS model

In [ ]:
embeddedData.loc[:, 'is-referenced-by-count'] = 0
predictionsDF = ar.relevancePredict(embeddedData, AWS = True)

In [ ]:
embeddedData

In [ ]:
predictionsDF.head()

In [ ]:
predictionsDF['prediction_date'] = datetime.now()
predictionsDF['prediction_date'] = predictionsDF['prediction_date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
predictionsDF['DOI'] = predictionsDF['DOI'].apply(str)

Save the columns we are interested on.

In [ ]:
predictionCols = ['DOI', 'prediction', 'predict_proba', 'model_metadata', 'prediction_date']

In [ ]:
predictionsDF

In [ ]:
predictionsDF = predictionsDF[predictionCols]

In [ ]:
predictionsDF['prediction'].value_counts()

In [ ]:
predictionsDF = predictionsDF[(predictionsDF['prediction'] == 1) |(predictionsDF['prediction'] == 0) ]

In [ ]:
predictionsDF.to_parquet('data/parquet/neotomaPredictions.parquet', index=False)